In [ ]:
!pip install transformers
# !pip install torch
# !pip install codecs


import torch
import itertools, random, pickle, codecs

import pandas as pd
import numpy as np
import os
from tqdm import tqdm

import torch.nn.functional as F
import torch.utils.data as D


In [2]:
from transformers import(
    AutoTokenizer,
    TextDataset,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments,
    AutoModelWithLMHead,
    AutoConfig,
    GPT2Config,
     pipeline,
      GPT2LMHeadModel,
      GPT2Tokenizer)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
DRIVE_PATH = "/content/drive/MyDrive/NLP/HW3"


In [32]:
from importlib.machinery import SourceFileLoader
ModelsModule = SourceFileLoader("ModelsModule", DRIVE_PATH+'/Models.py').load_module()

from ModelsModule import SiameseLSTM


In [ ]:
#download pretrained gpt2 model
!wget "https://huggingface.co/bolbolzaban/gpt2-persian/resolve/main/pytorch_model.bin" -P /content/gpt2/
!wget "https://huggingface.co/bolbolzaban/gpt2-persian/resolve/main/tokenizer.json" -P /content/gpt2/

In [ ]:
!cp -r "/content/gpt2" DRIVE_PATH+'/gpt2'   #save model in drive for future use, or not :D 

**Dataset**

In [262]:
mesras = list(filter(lambda m:len(m)>2, \
                     [x.strip().split() for x in codecs.open(DRIVE_PATH+f'/Persian_poems_corpus/normalized/ferdousi_norm.txt','rU','utf-8').readlines()]))


In [277]:
rhyming_beyts = []
non_rhyming_beyts = []

for idx in range(0, len(mesras)-1, 2):
  rhyming_beyts.append((' '.join(mesras[idx]), ' '.join(mesras[idx+1])))


dataset = []
for beyt in rhyming_beyts:
  dataset.append('[BOM] ' + (beyt[0])   + ' [BOM] '  + (beyt[1]) + ' [EOS]') 
# for beyt in non_rhyming_beyts:
#   dataset.append('[CLS]ندارد[CLS]' + '[BOM]' + (beyt[0])  +  '[BOM]' + (beyt[1]) + '[EOS]' )

In [281]:
random.shuffle(dataset)
train_dataset = dataset[:train_size]
val_dataset = dataset[:val_size]

# Split into training and validation sets
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

f'There are {len(train_dataset)} samples for training, and {len(val_dataset)} samples for validation testing'

'There are 44643 samples for training, and 4961 samples for validation testing'

In [282]:

with open(DRIVE_PATH+"/train.txt", "w") as output:
      for row in train_dataset:
            s = str(row)
            output.write(s+'\n')

with open(DRIVE_PATH+"/test.txt", "w") as output:
    for row in val_dataset:
            s = str(row)
            output.write(s+'\n')

In [5]:
# model_name = "HooshvareLab/gpt2-fa"
model_name = "bolbolzaban/gpt2-persian"
model_path = DRIVE_PATH+'/gpt2' # or /content/gpt2
trained_model_path = DRIVE_PATH+'/model'
train_path =  DRIVE_PATH+"/train.txt"
test_path =  DRIVE_PATH+"/test.txt"

**Training Model**


In [ ]:

tokenizer = AutoTokenizer.from_pretrained(model_name)
# special_tokens_dict = { 'additional_special_tokens': [ '[EOM]']}

# num_added_tokens = tokenizer.add_special_tokens(special_tokens_dict)

# config = AutoConfig.from_pretrained(
#     model_name)

# tokenizer.save_pretrained("/content/gpt2/")
# config.save_pretrained("/content/gpt2/")


In [ ]:
configuration = GPT2Config.from_pretrained(model_name)

model = AutoModelWithLMHead.from_pretrained(model_path , config = configuration)
model.resize_token_embeddings(len(tokenizer))


In [8]:
def freeze_lower_layers():
    for param in model.base_model.parameters():
        param.requires_grad = False

    for param in (
        model.base_model.h[23].parameters() or model.base_model.h[22].parameters()
    ):
        param.requires_grad = True


In [9]:
freeze_lower_layers()


In [10]:
def load_dataset(train_path, test_path, tokenizer):
    train_dataset = TextDataset(
        tokenizer=tokenizer, file_path=train_path, block_size=256
    )

    test_dataset = TextDataset(tokenizer=tokenizer, file_path=test_path, block_size=256)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False,
    )

    return train_dataset, test_dataset, data_collator


In [ ]:
train_dataset, test_dataset, data_collator = load_dataset(
    train_path, test_path, tokenizer
)

In [12]:
# epochs = 5
# sample_every = 500
# learning_rate = 3e-5
# eps = 1e-8
# warmup_steps = 500
training_args = TrainingArguments(
    output_dir=DRIVE_PATH+"/model",
    overwrite_output_dir=True,
    num_train_epochs=10,
    per_device_train_batch_size=12,
    per_device_eval_batch_size=12,
    eval_steps=1000,
    save_steps=1000,
    warmup_steps=500,
)
training_args.device

device(type='cuda', index=0)

In [ ]:
CUDA_LAUNCH_BLOCKING=1
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

trainer.train()

# save
trainer.save_model()


**Evaluation**

In [ ]:
del model

In [ ]:
model = GPT2LMHeadModel.from_pretrained(trained_model_path)
tokenizer = AutoTokenizer.from_pretrained(model_name)


In [153]:
model1 = GPT2LMHeadModel.from_pretrained(trained_model_path + '/checkpoint-eom')
tokenizer1 = AutoTokenizer.from_pretrained(model_name)
special_tokens_dict = { 'additional_special_tokens': [ '[EOM]']}

num_added_tokens = tokenizer1.add_special_tokens(special_tokens_dict)
model1.eval()

In [13]:
input = "[BOM] بی تو روزی از آن کوچه گذشتم [BOM]"

In [18]:
 generator(input)

Setting `pad_token_id` to `eos_token_id`:9 for open-end generation.


[{'generated_text': '[BOM] بی تو روزی از آن کوچه گذشتم [BOM] که ز یک کوچه ندیدیم نشانت'},
 {'generated_text': '[BOM] بی تو روزی از آن کوچه گذشتم [BOM] که به دیدار تو از دور رسیدم'},
 {'generated_text': '[BOM] بی تو روزی از آن کوچه گذشتم [BOM] بی تو شب بر سر آن کوچه گذشتم'},
 {'generated_text': '[BOM] بی تو روزی از آن کوچه گذشتم [BOM] که مرا خانه به دوشان گذر افتاد'},
 {'generated_text': '[BOM] بی تو روزی از آن کوچه گذشتم [BOM] که به اندازه یک عمر نشستم'}]

In [34]:
device = torch.device("cpu")

if torch.cuda.is_available():
   device = torch.device("cuda:0")

rhyme_model = SiameseLSTM(embedding_dim=128, hidden_dim=512, num_layers=1, dropout=0.0, learning_rate=0.01, device=device)

rhyme_model.load_state_dict(torch.load(DRIVE_PATH+'/rhyme_models/rhyme_model19000.pt', map_location=torch.device(device)))

rhyme_model.eval()


SiameseLSTM(
  (char_embedding): Embedding(36, 128)
  (lstm): LSTM(
    (lstm): LSTM(128, 512, batch_first=True)
  )
  (loss_fn): CosineEmbeddingLoss()
)

In [84]:
def do_rhyme_words(w1,w2):
    return (rhyme_model.predict(w1, w2) > 0.93)

def do_rhyme_mesras(m1, m2):
    if m1[-1] == m2[-1]:
      return do_rhyme_words( m1[-2], m2[-2] )
    else:
      return do_rhyme_words( m1[-1], m2[-1] )
      

In [74]:
import re
def generator(model, tokenizer, input, rhyme = True, max_length=20,  num_return_sequences=10):
    prompt = f"[BOM] {input} [BOM]"

    generator_pipline = pipeline("text-generation", model, tokenizer=tokenizer, do_sample = True,\
                    top_k = 50, top_p = 0.95, \
                    return_full_text = False, \
                    config={"max_length": max_length}, num_return_sequences = num_return_sequences)

    rhyming = []
    non_rhyming = []
  
    repeat = (rhyme and len(rhyming) == 0) or ((not rhyme) and len(non_rhyming) == 0)

    while(repeat):

      gen_outputs = generator_pipline(prompt)

      for  output in gen_outputs:
        out = output['generated_text']
        # print(out)
        out = out.replace("[BOM]", " ")
        if do_rhyme_mesras(input.strip().split() , out.strip().split() ):
              rhyming.append(out)              
        else:
              non_rhyming.append(out)
      
      repeat = (rhyme and len(rhyming) == 0) or (not rhyme and len(non_rhyming) == 0)


    print(f'Input: {input} \n')
    final_outputs = [rhyming if rhyme else non_rhyming]
    print(f'Generated outputs:\n')
    print(*final_outputs, sep = '\n')

    


In [69]:
input = 'بی تو روزی از آن کوچه گذشتم'
generator(model = model, tokenizer = tokenizer, input = input, rhyme = True)

Setting `pad_token_id` to `eos_token_id`:9 for open-end generation.


Input: بی تو روزی از آن کوچه گذشتم 

Generated outputs:

[' که دگر بار نیامد ز تو بویم', ' از تو ای عشق تو ای باده شنیدم', ' جز به دیدار تو ای دوست ندیدم', ' وز سر کوی تو رفتم', ' که به جز سایه دیوار ندیدم']


In [75]:
input = 'بی تو روزی از آن کوچه گذشتم'
generator(model = model, tokenizer = tokenizer, input = input, rhyme = False)

Setting `pad_token_id` to `eos_token_id`:9 for open-end generation.


Input: بی تو روزی از آن کوچه گذشتم 

Generated outputs:

[' که ز اندازه گذشتند و نجستند مرا', ' که مرا یاد نکردی چو گذشتی', ' رفت آن شب که ز تیمار تو دل بر دارم', ' تا نگویی که که این باغ تو را کیست', ' همچو سروی که بود سایه دیوار', ' به خیال تو که دل بسته به مویی', ' که ترا در بر و بی برگ و نوایی باشد']


In [81]:
input = 'گاه گویند دریا خاک شد'
generator(model = model, tokenizer = tokenizer, input = input, rhyme = True)

Setting `pad_token_id` to `eos_token_id`:9 for open-end generation.


Input: گاه گویند دریا خاک شد 

Generated outputs:

[' گه بگویند کشتی چاک شد', ' گاه گویند با خاک آمد', ' گاه گویند آتشش خاشاک شد', ' گه گویند گردون چاک شد']


In [89]:
# import re
# def generator(prompt, max_length=20,  num_return_sequences=10):
#     prompt = f"[BOM] {prompt} [BOM] "
#     print(prompt)
#     generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
#     # generated = generated.to(device)

#     decoded_outputs = model.generate(
#         generated,
#         do_sample=True,
#         top_k=50,
#         max_length=max_length, 
#         top_p=0.95,
#         num_return_sequences=num_return_sequences,
#         output_scores=True
#     )
#     print(decoded_outputs)
#     gen_sequences = decoded_outputs.sequences[:, generated.shape[-1]:]


#     probs = torch.stack(decoded_outputs.scores, dim=1).softmax(-1)  # -> shape [3, 15, vocab_size]

#     gen_probs = torch.gather(probs, 2, gen_sequences[:, :, None]).squeeze(-1)
#     normed_gen_probs = gen_probs / gen_probs.sum(0)

#     unique_normed_prob_per_sequence = normed_gen_probs.prod(-1)


#     for i, output in enumerate(gen_sequences):
#         o = tokenizer.decode(output, skip_special_tokens=True)
#         o = o.replace("[BOM]", " ")


#         print(o)
#         print(f'probability:{unique_normed_prob_per_sequence[i]}')
#         print('-' * 100)